In [ ]:
"""
Created on Fri Sep 09 14:24 2022

Convert "raw output" from the model to 2D maps - for the figure only tblocks 11, 12, 13 needed (OPM021 - REALISTIC)

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
import seaborn as sns
import datetime
import time
import os,sys

import tensorflow as tf
from tensorflow import keras
from contextlib import redirect_stdout

import nn_functions.data_formatting as dfmt
import nn_functions.postprocessing_functions as pp
from multimelt.constants import *

DEFINE OPTIONS

In [ ]:
mod_size =  'xsmall96' #'mini', 'small', 'medium', 'large', 'extra_large'
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'newbasic2' #'onlyTSdraftandslope' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

In [ ]:
tblock_dim = range(1,14)
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]

READ IN DATA

In [ ]:
inputpath_data_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'


In [ ]:
if TS_opt == 'extrap':
    inputpath_CVinput = inputpath_data_nn+'EXTRAPOLATED_ISFDRAFT_CHUNKS_CV/'
    path_orig_data = inputpath_data_nn+'EXTRAPOLATED_ISFDRAFT_CHUNKS/'
elif TS_opt == 'whole':
    inputpath_CVinput = inputpath_data_nn+'WHOLE_PROF_CHUNKS_CV/'
    path_orig_data = inputpath_data_nn+'WHOLE_PROF_CHUNKS/'
elif TS_opt == 'thermocline':
    inputpath_CVinput = inputpath_data_nn+'THERMOCLINE_CHUNKS_CV/'

APPLY MODEL

In [ ]:
input_list = ['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat','theta_in','salinity_in',
                  'T_mean', 'S_mean', 'T_std', 'S_std']

CV over time

In [ ]:
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/CV_TBLOCKS/'

res_all_list = []
nemo_run_old = 'OPM021'
#for tblock_out in tqdm(tblock_dim):
#for tblock_out in tqdm([1,2]):
for tblock_out in tqdm([11,12,13]):

    isf_out = 0

    nemo_run = pp.identify_nemo_run_from_tblock(tblock_out)

    if (nemo_run_old != nemo_run):
        print('I AM SAVING RESULTS FOR NEMO RUN '+nemo_run_old)
        res_all_CV = xr.concat(res_all_list, dim='time')
        res_all_CV.to_netcdf(outputpath_melt_nn + 'evalmetrics_2D_CV_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_'+nemo_run_old+'.nc')
        res_all_list = []
    
    nemo_run_old = nemo_run
        
    res_all = pp.compute_crossval_metric_2D_for_1CV(tblock_out,isf_out,tblock_dim,isf_dim,inputpath_CVinput,path_orig_data,norm_method,TS_opt,mod_size+'_'+exp_name,'experiments/',input_vars=input_list,verbose=False)
    res_all_list.append(res_all)
    
    if (tblock_out == max(tblock_dim)):
        print('I AM SAVING RESULTS FOR NEMO RUN '+nemo_run_old)
        res_all_CV = xr.concat(res_all_list, dim='time')
        res_all_CV.to_netcdf(outputpath_melt_nn + 'evalmetrics_2D_CV_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_'+nemo_run_old+'.nc')

CV over shelves

In [ ]:
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/CV_ISF/'

t_list = []
res_all_list = None
#nemo_run_old = 'OPM006'
nemo_run_old = 'OPM021'

#for tt in tblock_dim:
for tt in [11,12,13]:
    nemo_run = pp.identify_nemo_run_from_tblock(tt)
    print(nemo_run)
    
    if (nemo_run_old != nemo_run):
        print(t_list)
        
        for isf_out in tqdm(isf_dim):
        #for isf_out in tqdm([44,66]):
            
            tblock_out = 0
        
            res_all = pp.compute_crossval_metric_2D_for_1CV(tblock_out,isf_out,t_list,isf_dim,inputpath_CVinput,path_orig_data,norm_method,TS_opt,mod_size+'_'+exp_name,'experiments/',input_vars=input_list,verbose=False)
            if res_all_list is None:
                res_all_list = res_all
            else:
                res_all_list = res_all_list.combine_first(res_all)
            
        res_all_CV = res_all_list
        print('I AM SAVING RESULTS FOR NEMO RUN '+nemo_run_old)
        res_all_CV.to_netcdf(outputpath_melt_nn + 'evalmetrics_2D_CV_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_'+nemo_run_old+'.nc')
        res_all_list = None
        
        t_list = []
        
    t_list.append(tt)
    nemo_run_old = nemo_run
    
    if (tt == max(tblock_dim)):
        print(t_list)
        
        for isf_out in tqdm(isf_dim):
            
            tblock_out = 0
        
            res_all = pp.compute_crossval_metric_2D_for_1CV(tblock_out,isf_out,t_list,isf_dim,inputpath_CVinput,path_orig_data,norm_method,TS_opt,mod_size+'_'+exp_name,'experiments/',input_vars=input_list,verbose=False)
            if res_all_list is None:
                res_all_list = res_all
            else:
                res_all_list = res_all_list.combine_first(res_all)
            
        res_all_CV = res_all_list
        print('I AM SAVING RESULTS FOR NEMO RUN '+nemo_run_old)
        res_all_CV.to_netcdf(outputpath_melt_nn + 'evalmetrics_2D_CV_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_'+nemo_run_old+'.nc')

            